In [1]:
!pip3 install collatex lxml Halo joblib pandas python-Levenshtein ordered_set json_minify dicttoxml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for lxml from https://files.pythonhosted.org/packages/3c/d2/11533f0bc47ff4d828a20cfb702f3453fe714bd5b475fcdc8cec6e6b7dcf/lxml-4.9.3-cp310-cp310-manylinux_2_28_x86_64.whl.metadata
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for joblib from https://files.pythonhosted.org/packages/10/40/d551139c85db202f1f384ba8bcf96aca2f329440a844f924c8a0040b6d02/joblib-1.3.2-py3-none-any.whl.metadata
  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/b1/67/aca1f6e215d957d24d0a290321f368503305480268f9617bf625243e9dea/pandas-2.1.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for python-Levenshtein from https://files.pythonhosted.org/packages/27/89/c45dbdbd479453cfc8c4c1271c9f67fd607deaf84880e31c67b682980456/python_Levenshtein

In [25]:
import sys
sys.path.insert(1, 'python/collation/')
sys.path.insert(1, 'python/injections/')
sys.path.insert(3, 'python/')
sys.path.insert(4, 'python/utils/')
sys.path.insert(4, 'python/sorties/')
import collation
import injections
import settings
import utils.utils as utils
import collator
import sorties
import re
import glob

On commence par importer les paramètres

In [19]:
correction = False
saxon = "saxon9he.jar"
deplacements = False
log = correction
fichier_de_parametres = "settings.json"
witness = "*"
division = 22
# On importe les paramètres en créant un objet parametres
parametres = settings.Parametres(fichier_de_parametres)
alignement = parametres.alignement
temoin_base = parametres.temoin_base
parametres.alignement = "mam"
align_on = parametres.align_on
synonyms_datasets = parametres.create_synonym_dataset
compute_similarity = parametres.compute_similarity
chemin_corpus = parametres.corpus_path
xpath_transcriptions = parametres.files_path
liste_sigles = utils.sigles()
excluded_ancestors = parametres.exclude_descendant_of
liste_fichiers_tokenises = utils.chemin_temoins_tokenises()

In [ ]:
# We first remove all files in the corresponding dir to avoid any possible interference and bug
utils.remove_files(f"divs/div{str(i)}/*")

chemin_fichiers = f"divs/div{str(i)}"
print(f"Traitement de la division {str(i)}")

In [ ]:
# On prépare le corpus pour la collation
corpus_preparator = collation.CorpusPreparation(saxon=saxon,
                                                temoin_leader=parametres.temoin_leader,
                                                type_division=parametres.type_division,
                                                element_base=parametres.element_base,
                                                liste_temoins=utils.chemin_temoins_tokenises_regularises(),
                                                integrer_deplacements=deplacements)


corpus_preparator.prepare(i)
pattern = re.compile(f"divs/div{i}/juxtaposition_\d+\.xml")
fichiers_xml = [fichier.split('/')[-1] for fichier in glob.glob(f"{chemin_fichiers}/*.xml") if
                re.match(pattern, fichier)]
print("Alignement avec CollateX.")

aligner = collation.Aligner(liste_fichiers_xml=fichiers_xml,
                            chemin=chemin_fichiers,
                            moteur_transformation_xsl=saxon,
                            correction_mode=correction,
                            align_on=align_on,
                            parametres_alignement=parametres.alignement,
                            nombre_de_coeurs=parametres.parallel_process_number)
aligner.run()

Le résultat est un ensemble de divisions alignées (celles indiquées dans le fichier de paramètres)

In [ ]:
# On va fusionner les fichiers individuels collationnés en un seul fichier.
chemin_fichier_json = f"{chemin_fichiers}/final.json"
with open(chemin_fichier_json, "w") as out_json_file:
    dictionnaire_sortie = {'table': [], 'witnesses': []}
    par_nb = len(glob.glob(f"{chemin_fichiers}/alignement_collatex*.json"))  # on veut ordonner la
    # fusion des
    # documents pour le tableau d'alignement ensuite
    for par in range(par_nb):
        fichier = f"{chemin_fichiers}/alignement_collatex{par + 1}.json"
        with open(fichier, 'r') as file:
            dictionnaire_entree = json.loads(file.read())
            nombre_temoins = len(dictionnaire_entree['table'])  # nombre_temoins est le nombre de témoins
            for index_temoin in range(nombre_temoins):  # pour chaque témoin
                temoin = dictionnaire_entree['witnesses'][index_temoin]
                if len(dictionnaire_sortie[
                           'witnesses']) != nombre_temoins:  # tant que la liste des témoins nombre_temoins'est pas complète
                    dictionnaire_sortie['witnesses'].append(temoin)
                if len(dictionnaire_sortie['table']) != nombre_temoins:  #
                    liste_vide = []
                    dictionnaire_sortie['table'].append(liste_vide)
                for element in dictionnaire_entree['table'][index_temoin]:
                    dictionnaire_sortie['table'][index_temoin].append(element)
    json.dump(dictionnaire_sortie, out_json_file)

In [ ]:
# On compare les lieux variants et on en déduit les <app>
with Halo(text='Création des apparats', spinner='dots'):
    # Étape suivante: transformer le JSON en xml. Pour cela on peut utiliser dict2xml.
    chemin_alignement = f"{chemin_fichiers}/alignement_collatex.xml"
    with open(chemin_alignement, "w+") as sortie_xml:
        with open(chemin_fichier_json, 'r') as fichier_json_a_xmliser:
            obj = json.loads(fichier_json_a_xmliser.read())
            vers_xml = dicttoxml.dicttoxml(obj).decode("utf-8")
        sortie_xml.write(vers_xml)

    chemin_regroupement = "xsl/post_alignement/regroupement.xsl"
    # Regroupement des lieux variants (témoin A puis témoin B puis témoin C
    # > lieu variant 1: A, B, C ; lieu variant 2: A, B, C)
    cmd = f"java -jar {saxon} -o:{chemin_fichiers}/aligne_regroupe.xml {chemin_fichiers}/alignement_collatex.xml " \
          f"{chemin_regroupement}"
    subprocess.run(cmd.split())

    # C'est à ce niveau que l'étape de correction devrait avoir lieu. Y réfléchir.
    # Création de l'apparat: transformation de aligne_regroupe.xml en JSON
    chemin_xsl_apparat = "xsl/post_alignement/creation_apparat.xsl"
    cmd = f"java -jar {saxon} -o:{chemin_fichiers}/apparat_final.json {chemin_fichiers}/aligne_regroupe.xml " \
          f"{chemin_xsl_apparat}"
    subprocess.run(cmd.split())
    # Création de l'apparat: suppression de la redondance, identification des lieux variants,
    # regroupement des lemmes
    

In [26]:
# Création du tableau d'alignement pour visualisation
if parametres.tableauxAlignement:
    sorties.tableau_alignement(saxon, chemin_fichiers)
    

AttributeError: module 'sorties' has no attribute 'tableau_alignement'

L'alignement a fonctionné: la collation peut commencer.

In [ ]:
collationeur = collation.Collateur(log=False,
                                   chemin_fichiers=chemin_fichiers,
                                   div_n=division,
                                   div_type=parametres.type_division,
                                   temoin_base=temoin_base)
collationeur.run_collation()
print("Création des apparats ✓")

In [ ]:
# On bouge tous les fichiers d'alignement dans un dossier à part
fichiers_alignement = glob.glob(f"{chemin_fichiers}/align*")
fichiers_alignement.extend(glob.glob(f"{chemin_fichiers}/juxtaposition*"))
fichiers_alignement.append(f"{chemin_fichiers}/apparat_final.json")
fichiers_alignement.append(f"{chemin_fichiers}/apparat_collatex.xml")
utils.move_files(fichiers_alignement, f"{chemin_fichiers}/alignement")

La collation terminée, on a une table de collation réinjectée dans les XML-TEI. La suite est le transfert des informations décidées dans le fichier de paramètres.

In [ ]:
injecteur = injections.Injector(debug=True,
                                div_n=i,
                                elements_to_inject=parametres.reinjection,
                                saxon=saxon,
                                chemin=chemin_fichiers,
                                coeurs=parametres.parallel_process_number,
                                element_base=parametres.element_base,
                                type_division=parametres.type_division,
                                lacuna_sensibility=parametres.lacuna_sensibility,
                                liste_sigles=liste_sigles,
                                excluded_elements=excluded_ancestors,
                                temoin_base=temoin_base)
injecteur.run_injections()
# Ici on indique d'autres éléments tei à réinjecter.

# On copie les fichiers finaux produits pour ne pas avoir à refaire à chaque fois le processus
for file in glob.glob(f"{chemin_fichiers}/*_injected_punct.transposed.lacuned.xml"):
    shutil.copy(file, f'divs/results')

Nettoyages divers

In [ ]:
liste_fichiers_finaux = utils.chemin_fichiers_finaux(i)
print("Cleaning files, producing final documents")
for file in glob.glob(f"divs/div{i}/*transposed.xml"):
    sigle = utils.get_sigla_from_path(file)
    print(sigle)
    utils.clean_xml_file(input_file=file, output_file=f"divs/div{i}/apparat_{sigle}_{i}_final.xml")

for file in glob.glob(f"div{chemin_fichiers}/*final.xml"):
    shutil.copy(file, f'divs/results')

sorties.nettoyage("divs")

Calcul du temps de computation

In [ ]:
t1 = time.time()
temps_total = t1 - t0
print(f"Fait en {round(temps_total)} secondes. \n")

Traitement de la division 22
Scission du corpus, création de dossiers et de fichiers par chapitre sur 22.

Préparation du corpus pour alignment ✓
Alignement avec CollateX.
